# Lab 5 - Localization

## Part A - Writing Assignment

<a id='question1'></a>
**Question 1**. (**Motion Model**) Consider a deterministic motion model (no added noise) based on odometry information. The motion model, $g$, takes as arguments the old particle pose, $\mathbf{x}_{t-1}$, as well as the current odometry data, $\Delta\mathbf{x}$, and returns a new pose, $\mathbf{x}_t$ with the odometry “applied” to the old poses:

$$
\begin{align}
    \mathbf{x}_t = \begin{bmatrix}x_t\\y_t\\\theta_t\end{bmatrix} = g(\mathbf{x}_{t-1}, \Delta x)
\end{align}
$$

Note that the poses $\mathbf{x}$ are expressed in the world frame relative to some arbitrary origin. The odometry $\Delta \mathbf{x}$ is necessarily a local quantity, expressed relative to the previous frame.

**i** Suppose you are given $\mathbf{x}_{t-1} = \left[0, 0, \frac{\pi}{6}\right]^T$ and $\mathbf{x}_{t} = \left[0.2, 0.1, \frac{11\pi}{60}\right]^T$. Compute the odometry data $\Delta \mathbf{x}$ that results in this transformation.

**ii**. Now suppose you received the odometry data $\Delta\mathbf{x}$ from part **i**, but your car was previously at position $\mathbf{x}_{t-1} = \left[3, 4, \frac{\pi}{3}\right]^T$. Compute the current pose $\mathbf{x}_t$.

If you were to use this deterministic motion model in your particle filter all of your particles would end up in the same place - which defeats the purpose of having so many. When you build your actual motion model, you will be injecting noise into the function $g$ which will make your particles spread out as the car moves. This accounts for any uncertainty that exists in the odometry information. The sensor model will collapse this spreading of particles back into a small region.

### Solution
i. The estimated change of pose $\Delta \mathbf{x}$ is actually the pose at time $t$ in the body frame at time $t-1$.

The transform from body frame to world frame $T^{B}_{W}$, is equal to the inverse of the vehicle's pose in the world frame $\mathbf{x}^{W}_{t-1} = <x_{t-1}, y_{t-1}, \theta_{t-1}>$. 

$$
T^{B}_{W} = (T^{W}_{B})^{-1} = 
\begin{bmatrix}
\mathbf{R}_{t-1} & \mathbf{P}_{t-1}\\
\mathbf{0} & 1
\end{bmatrix}^{-1}
=
\begin{bmatrix}
\mathbf{R}^{-1}_{t-1} & -\mathbf{R}_{t-1}^{-1}\mathbf{P}_{t-1}\\
\mathbf{0} & 1
\end{bmatrix}
=
\begin{bmatrix}
\cos\theta_{t-1} & \sin\theta_{t-1} & -(x_{t-1}\cos\theta_{t-1} + y_{t-1}\sin\theta_{t-1}) \\
-\sin\theta_{t-1} & \cos\theta_{t-1} & -(-x_{t-1}\sin\theta_{t-1} + y_{t-1}\cos\theta_{t-1}) \\
0 & 0 & 1
\end{bmatrix}
$$

$T^{B}_W$ brings pose at time $t$ in world frame to body frame:
$$
\Delta \mathbf{x}^{B} = T_{W}^{B} \cdot \mathbf{x}_{t}^{W} = \begin{bmatrix}
\cos\theta_{t-1} & \sin\theta_{t-1} & -(x_{t-1}\cos\theta_{t-1} + y_{t-1}\sin\theta_{t-1}) \\
-\sin\theta_{t-1} & \cos\theta_{t-1} & -(-x_{t-1}\sin\theta_{t-1} + y_{t-1}\cos\theta_{t-1}) \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
\cos\theta_t & -\sin \theta_t & x_t\\
\sin\theta_t & \cos \theta_t & y_t \\
0 & 0 & 1
\end{bmatrix}
$$

Plug in the values provided and convert resulting matrix to vector form, we find $\Delta \mathbf{x}^{B} = <0.223, -0.0134, 0.0524>$

In [2]:
import numpy as np
from math import sqrt, pi, cos, sin, atan2, atan 

# ----------- help function --------------------
def rotation_matrix(theta):
    return np.array([[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])

def dist(dx, dy):
    return sqrt(dx*dx + dy*dy)

def pose_matrix_from_vec(pose_vec):
    x = pose_vec[0]
    y = pose_vec[1]
    theta = pose_vec[2]
    return np.array([[cos(theta), -sin(theta), x], [sin(theta), cos(theta), y], [0, 0, 1]])
# ----------- help function --------------------


'''
finddx:
    Calculate deltas between consecutive odometry messages in the coordinate space of the car.
    
Three possible solutions are given below.
'''
def finddx_pbook(last_pose, cur_pose):
    '''
    Solution from Probabilistic Robotics
    '''
    assert(isinstance(last_pose, np.ndarray))
    
    dx = cur_pose[0] - last_pose[0]
    dy = cur_pose[1] - last_pose[1]

    delta_rot1 = atan2(dy, dx) - last_pose[2]
    delta_trans = dist(dx, dy) 
    delta_rot2 = cur_pose[2] - last_pose[2] - delta_rot1

    return np.array([delta_trans * cos(delta_rot1), delta_trans * sin(delta_rot1), delta_rot1+delta_rot2])

def finddx_2018(last_pose, cur_pose):
    '''
    Solution from previous year
    '''
    assert(isinstance(last_pose, np.ndarray))
    
    rot = rotation_matrix(-last_pose[2])
    delta = np.array([cur_pose[0:2]- last_pose[0:2]]).transpose()
    local_delta = (np.dot(rot,delta)).transpose()
    assert(local_delta.shape == (1, 2))
    return np.array([local_delta[0,0], local_delta[0,1], cur_pose[2]- last_pose[2]])

def finddx_matrix(last_pose, cur_pose):
    '''
    Solution using transform matrix
    '''
    assert(isinstance(last_pose, np.ndarray))
    assert(isinstance(cur_pose, np.ndarray))
    
    T_BW = np.linalg.inv(pose_matrix_from_vec(last_pose))
    dx_W = pose_matrix_from_vec(cur_pose)

    res = np.dot(T_BW, dx_W)
    delta_theta_B = atan(res[1][0]/res[0][0]) 

    return np.array([res[0][2], res[1][2], delta_theta_B]) 

last_pose = np.array([0.0, 0.0, pi/6.0])
cur_pose = np.array([0.2, 0.1, pi * 11.0 / 60.0])


print('Code from last year: odom_msg:', finddx_2018(last_pose, cur_pose))
print('Algorithm from p-book: odom_msg', finddx_pbook(last_pose, cur_pose))
print('Code using transform matrix: odom_msg', finddx_matrix(last_pose, cur_pose))

Code from last year: odom_msg: [ 0.22320508 -0.01339746  0.05235988]
Algorithm from p-book: odom_msg [ 0.22320508 -0.01339746  0.05235988]
Code using transform matrix: odom_msg [ 0.22320508 -0.01339746  0.05235988]


ii. 
Again, we can easiy find the updated pose from transform matrix. 
$$
\mathbf{x}^{W}_t = T^{W}_{B} \cdot \Delta\mathbf{x}^B
$$

In [16]:
odom_msg = finddx_pbook(last_pose, cur_pose)
x_prev = np.array([3, 4, pi/3.0])

'''
Odometry data is accumulated via dead reckoning, so it is very inaccurate on its own.
We apply the pose update from odometry measurement on the pose (estimated from last time step)

Three possible solutions are given below.
'''
def apply_odom_pbook(x_prev, odom_msg):
    dx = odom_msg[0]
    dy = odom_msg[1]
    dtheta = odom_msg[2]
    dr = sqrt(dx*dx + dy*dy)
    
    delta_rot1 = atan2(dy, dx)
    xt = x_prev[0] + dr * cos(x_prev[2] + delta_rot1)
    yt = x_prev[1] + dr * sin(x_prev[2] + delta_rot1)
    thetat = x_prev[2] + dtheta
    
    return np.array([xt, yt, thetat])
   
def apply_odom_2018(x_prev, odom_msg):
    cosines = np.cos(x_prev[2])
    sines = np.sin(x_prev[2])

    local_deltas = np.zeros(3)
    local_deltas[0] = cosines*odom_msg[0] - sines*odom_msg[1]
    local_deltas[1] = sines*odom_msg[0] + cosines*odom_msg[1]
    local_deltas[2] = odom_msg[2]
    
    return x_prev + local_deltas

def apply_odom_matrix(x_prev, odom_msg):
    x_t = np.dot(pose_matrix_from_vec(x_prev), pose_matrix_from_vec(odom_msg))
    
    return np.array([x_t[0][2], x_t[1][2], atan(x_t[1][0]/x_t[0][0])])

print('x_t in world frame (pbook):', apply_odom_pbook(x_prev, odom_msg))
print('x_t in world frame (code from last year):', apply_odom_2018(x_prev, odom_msg))
print('x_t in world frame (transform matrix):', apply_odom_matrix(x_prev, odom_msg))

x_t in world frame (pbook): [3.12320508 4.18660254 1.09955743]
x_t in world frame (code from last year): [3.12320508 4.18660254 1.09955743]
x_t in world frame (transform matrix): [3.12320508 4.18660254 1.09955743]


<a id='question2'></a>
**Question 2**. (**Sensor Model**) The sensor model, $p\left(z_{t}| x_{t}, m\right)$, defines how likely it is to record a given sensor reading $z_{t}$ from a hypothesis position $x_{t}$ in a known, static map $m$ at time $t$. We will use this likelihood to "prune" the particles as the motion model tries to spread them out. Particles will be kept if the sensor reading is consistent with the map from the point of view of that particle. The definition of this likelihood is strongly dependent on the type of sensor used - a laser scanner in our case.

In a lidar model, typically, there are a few cases to be modeled in determining $p\left(z_{t}| x_{t}, m\right)$:

1. Probability of detecting a known obstacle in the map
2. Probability of a short measurement. Maybe due to internal lidar reflections (scratches or oils on the surface), hitting parts of the vehicle itself, or other unknown obstacles (people, cats, etc).
3. Probability of a very large (aka missed) measurement. Usually due to lidar beams that hit an object with strange reflective properties and did not bounce back to the sensor
4. Probability of a completely random measurement. Just in case of an asteroid strike.
    
We typically represent (1) with a gaussian distribution centered around the ground truth distance between the hypothesis pose and the nearest map obstacle. Thus, if the measured range exactly matches the expected range, the probability is maximum. If the measured range is $z_{t}$ and the ground truth range is determined (via ray casting on the map $m$ from pose $x_t$) to be $z_{t}^*$ then we have that:

$$
	p_{hit}(z_{t}| x_{t}, m)  = \begin{cases}
	\eta \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(z_t - z_t^*)^2}{2\sigma^2}\right)  &   \text{if} \quad 0 \leq z_{t}^{k} \leq z_{max}\\
    0   &   \text{otherwise}
    \end{cases}
$$
where $\eta$ is a normalization constant need to be determined.

Case (2) is represented as a downward sloping line as the ray gets further from the robot.
This is because if the unknown obstacles (people cats, etc.) are distributed uniformly in the environment, the lidar is more likely to hit ones that are closer (think about how area scales...). This likelihood can be modeled as:

$$
	p_{short}\left(z_{t}| x_{t}, m\right) =  \frac{2}{z_t^*} \begin{cases}
         1 - \frac{z_{t}}{z_{t}^*}   &   \text{if} \quad 0 \leq z_{t}^{k} \leq z_{t}^*\\
         0   &   \text{otherwise}
\end{cases}
$$

Case (3) is represented by a large spike in probability at the maximal range value, so that reflected measurements do not significantly discount particle weights.

$$
	p_{max}(z_{t}| x_{t}, m)  =\begin{cases}
	1  &  \text{if} \quad z_t = z_{max}\\
	0  &  \text{otherwise} 
	\end{cases}
	$$

Case (4) is represented by a small uniform value, to account for unforeseen effects.

$$
	p_{rand}(z_{t}| x_{t}, m)  = \begin{cases}
	\frac{1}{z_{max}}  &  \text{if} \quad 0\leq z_{t} < z_{max}\\
	0                            & \text{otherwise} 
	\end{cases}
$$
	
These four different distributions are now mixed by a weighted average, defined by the parameters $\alpha_{hit}, \alpha_{short},\alpha_{max},\alpha_{rand}$:

$$
	 p(z_{t}| x_{t}, m)  = \alpha_{hit} \cdot p_{hit}(z_{t}| x_{t}, m)  + \alpha_{short} \cdot p_{short}(z_{t}| x_{t}, m)  + \alpha_{max} \cdot p_{max}(z_{t}| x_{t}, m)  + \alpha_{rand} \cdot p_{rand}(z_{t}| x_{t}, m) 
$$

Note that in order for $p(z_{t}| x_{t}, m)$ to be a probability distrubution we have have that:

$$
\alpha_{hit}+\alpha_{short}+\alpha_{max}+\alpha_{rand}=1
$$

	 
Find the values of $p(z_{t} | x_{t}, m)$ for the values of $z_t$ below and the following parameters:
$\alpha_{hit} = 0.74$,
$\alpha_{short}=0.07$,
$\alpha_{max}=0.07$,
$\alpha_{rand}=0.12$,
$\sigma=0.5\text{ m}$
$z_{max}=10\text{ m}$, and
$z_{t}^{*} = 7\text{ m}$.

**i**.    $z_{t} = 0\text{ m}$

**ii**.   $z_{t} = 3\text{ m}$

**iii**.  $z_{t} = 5\text{ m}$

**iv**.   $z_{t} = 8\text{ m}$

**v**.    $z_{t} = 10\text{ m}$

**Solution**

The required values $p(z_t|x_t, m)$ are probability density function (pdf, for continuous variables), we need to convert them to probability values in the precomupted table. Interestingly, if we work on the 'table space', the discretized size will be 1, and the values of pdf are equal to the values of probability.

See the `sensor_model.py` file for a completed version.

In [4]:
def sensor_model(d, r):
    '''
    The sensor model. For each discrete computed range value, this provides the probability 
    of measuring any (discrete) range.
    
    Argc:
        d: computed range from RangeLibc
        r: the observed range from the lidar unit
    Return:
        prob: the probability of observing r 
            conditioning on computed range and predefined map
    '''
    # sensor model constants
    alpha_hit = 0.74
    alpha_short = 0.07
    alpha_max = 0.07
    alpha_rand = 0.12
    
    sigma_hit = 0.5 
    c_r = 0.01
    
    MAX_RANGE_METER = 10
    # MAX_RANGE_PX = 200
    
    prob = 0.0
    z = float(r-d)
    # reflects from the intended object
    # nomalization constant \eta is approximated as 1 in this case
    prob +=  alpha_hit * np.exp(-(z*z)/(2.0*sigma_hit*sigma_hit)) / (sigma_hit * np.sqrt(2.0*np.pi))

    # observed range is less than the predicted range - short reading
    if r < d:
        prob += alpha_short * 2.0/d * (d - r) / float(d)

    # erroneous max range measurement
    if int(r) == int(MAX_RANGE_METER):
        prob += alpha_max

    # random measurement
    if r < int(MAX_RANGE_METER):
        prob += alpha_rand * 1.0/float(MAX_RANGE_METER)

    return prob

# d is the computed range from RangeLibc
d = 7 
# r is the observed range from the lidar unit
rlist = [0, 3, 5, 8, 10]
for r in rlist:
    prob = sensor_model(d, r)
    print('r: %f' % r, 'pdf: %f'% prob)
    
# Visualization the pdf
MAX_RANGE_METER = 10
rmin = 0
rmax = MAX_RANGE_METER
num = 1000
rlist = np.linspace(rmin, rmax, num)
pdf = [sensor_model(d, r) for r in rlist]

# import matplotlib.pyplot as plt
# plt.plot(rlist, pdf)
# plt.show()

r: 0.000000 pdf: 0.032000
r: 3.000000 pdf: 0.023429
r: 5.000000 pdf: 0.017912
r: 8.000000 pdf: 0.091907
r: 10.000000 pdf: 0.070000
